In [6]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install psycopg2

  Using cached https://files.pythonhosted.org/packages/84/d7/6a93c99b5ba4d4d22daa3928b983cec66df4536ca50b22ce5dcac65e4e71/psycopg2-2.8.4.tar.gz
anceled
ERROR: Operation cancelled by user


In [1]:
import pandas as pd
import psycopg2

In [2]:

connection = psycopg2.connect(
    host = 'localhost',
    port = 5432,
    user = 'postgres',
    password = 'iloveindia',
    database='titanic'
    )

In [3]:
#creating coursor object
cursor=connection.cursor()

In [9]:
#creating tables 
cursor.execute("""CREATE TABLE passengers(
id SERIAL PRIMARY KEY,
name text,
sex text,
age float,
sibsp integer,
parch integer)""")

cursor.execute("""CREATE TABLE survival(
id SERIAL PRIMARY KEY,
survived integer)""")

cursor.execute("""CREATE TABLE tripInfo(
id SERIAL PRIMARY KEY,
pclass integer,
ticket text,
fare float,
cabin text,
embarked text)""")

connection.commit()

DuplicateTable: relation "passengers" already exists


In [4]:
#Quering Information Shema table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,passengers,id,integer,public
1,passengers,sex,text,public
2,passengers,name,text,public
3,passengers,sibsp,integer,public
4,passengers,parch,integer,public
5,passengers,age,double precision,public
6,survival,id,integer,public
7,survival,survived,integer,public
8,titanic,embarked,text,public
9,titanic,survived,integer,public


In [6]:
# Copy data from csv files to tables:

with open('passengers.csv', 'r') as row:
    next(row)# Skip the header row.
    cursor.copy_from(row, 'passengers', sep=',')
    
with open('survival.csv', 'r') as row:
    next(row) 
    cursor.copy_from(row, 'survival', sep=',')

with open('trip_info.csv', 'r') as row:
    next(row)
    cursor.copy_from(row, 'tripInfo', sep=',')
    
connection.commit()   



In [7]:
# Create a new Role gastudent
cursor.execute("""CREATE ROLE gastudent WITH LOGIN PASSWORD 'gastudent'""")
#Grant different permissions for a new role
cursor.execute("""GRANT CONNECT ON DATABASE titanic TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON passengers TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON survival TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON tripInfo TO gastudent""")

connection.commit()

In [5]:
# simple query to interact with a remote DB.

sql = """
SELECT pclass, AVG(survived)
FROM (SELECT t.pclass, s.survived
FROM tripInfo t
INNER JOIN survival s ON t.id = s.id
) AS x
GROUP BY pclass
"""

pd.read_sql(sql, con=connection)

,pclass,avg
0,1,0.629630
1,3,0.242363
2,2,0.472826
